In [3]:
import pickle
import numpy as np
from tensorflow import keras

In [4]:
test_data = None
with open("../data/test_data.pickle", "rb") as f:
    test_data = pickle.load(f)

In [5]:
x_test = test_data['test_texts']
y_test = test_data['test_labels']

In [6]:
def get_chunks(l, n):
    n = max(1, n)
    return [l[i:i+n] for i in range(0, len(l), n)]

In [7]:
# 182 words is quite short
# Try to join 5 tests texts together
longer_test_texts = get_chunks(x_test, 5)
longer_test_labels = get_chunks(y_test, 5)

In [8]:
longer_test_texts = ['\n'.join(chunk) for chunk in longer_test_texts]
longer_test_labels = [chunk[0] for chunk in longer_test_labels]

In [11]:
author_models = []
for i in range(50):
    author_models.append((keras.models.load_model('../models/gru_models/author'+str(i)+'.h5'), i))

In [17]:
# vectorization - chars to ints
import string
import random
import sys

import numpy as np

from tensorflow import keras
# keras.models import load_model

def sample(preds, temperature=1.0):
    """Sample predictions from a probability array"""
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds + 1e-6) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def generate(model, diversity=0.5, text=""):
    """Generate text from a model"""
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated = ''
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(5000):
        x = np.zeros((1, maxlen), dtype=np.int)
        for t, char in enumerate(sentence):
            try:
                x[0, t] = char_indices[char]
            except:
                print(sentence)
        preds = model.predict(x, verbose=0)[0][0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]
        generated += next_char
        sentence = sentence[1:] + next_char
        sys.stdout.write(next_char)
        sys.stdout.flush()
    return

def vectorize(text):
    """Convert text into character sequences"""
    step = 3
    sentences = []
    next_chars = []
    for i in range(0, len(text) - maxlen, step):
        sentences.append(text[i: i + maxlen])
        next_chars.append(text[i + maxlen])
    X = np.zeros((len(sentences), maxlen), dtype=np.int)
    y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
    for i, sentence in enumerate(sentences):
        for t, char in enumerate(sentence):
            X[i, t] = char_indices[char]
        y[i, char_indices[next_chars[i]]] = 1
    return X, y

def clean_text(text, charset):
    text = " ".join(text.split())  # all white space is one space
    text = "".join([x for x in text if x in charset])  # remove characters that we don't care about
    return text

def get_model(modelfile, freeze=False):
    model = keras.models.load_model(modelfile)
    if freeze:
        for layer in model.layers[:6]:
            layer.trainable = False
    return model

chars = " " + string.ascii_letters + string.punctuation  # sorted to keep indices consistent
charset = set(chars)  # for lookup
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

maxlen = 100  # must match length which generated model - the sequence length


In [35]:
# indicies = list(range(len(longer)))
test_texts = np.array(longer_test_texts)
test_labels = np.array(longer_test_labels)

text = test_texts[3]
label = test_labels[3]
X, y = vectorize(clean_text(text, charset))


In [36]:
from datetime import datetime
predictions = []
losses = []
t1 = datetime.now()
for am in author_models:
    print(".", end="")
    model = am[0]
    label = am[1]
    loss = model.evaluate(X, y, verbose=0)
    losses.append((loss, label))
print(" {}".format(datetime.now() - t1))
predictions.append(losses)

.................................................. 0:02:53.166296


In [27]:
pred_is = []
for pred in predictions:
    pred_i = [p[0] for p in pred]
    pred_is.append(pred_i)
pred = [np.argmin(pred) for pred in pred_is]

In [31]:
pred

[0]

In [34]:
test_labels[0]

0